In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from datetime import datetime


In [15]:
def process_single_image(image_path, mask_path):
    """Process a single image using the original logic, but also consider a mask."""
    # Read the image with unchanged flag
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if image is None:
        return None, None, 0
    
    # Convert to 3-channel if it's single channel
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    
    # Read the mask image
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    
    # Create a new image
    new_image = np.zeros(image.shape, image.dtype)
    alpha = 3.5
    beta = 0
    
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            for c in range(image.shape[2]):
                new_image[y,x,c] = np.clip(alpha*image[y,x,c] + beta, 0, 255)
                
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    # threshold for arsenic
    threshold = 70
     
    mask = gray_image > threshold
    new_image[mask] = [0, 255, 0]  # Set to green
    
    # Convert to RGB for processing
    image_rgb = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
    hsv_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
    
    # Define range for green color in HSV
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
    
    # Find contours, but only outside the mask
    contours, * = cv2.findContours(green_mask & ~cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw bounding squares on the original RGB image
    annotated_image = image_rgb.copy()
    
    dot_info = []
    for i, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(annotated_image, (x, y), (x + w, y + h), (255, 255, 0), 2)
        
        dot_info.append({
            'dot_id': i + 1,
            'position': {'x': int(x + w/2), 'y': int(y + h/2)},
            'width': w,
            'height': h,
            'area': int(cv2.contourArea(contour))
        })
    
    # Convert back to BGR for saving
    annotated_image_bgr = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
    
    return annotated_image_bgr, dot_info, len(contours)

In [16]:
def process_folder(input_folder, output_folder, json_output_path, label, nucleus_mask_folder):
    """Process all images in a folder and save results."""
    os.makedirs(output_folder, exist_ok=True)
    
#     results = {
#         'processing_date': datetime.now().isoformat(),
#         'total_images_processed': 0,
#         'images': {}
#     }
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.tif', '.tiff')):
            image_path = os.path.join(input_folder, filename)
            print(f"Processing {filename}...")
            
            try:
                annotated_image, dot_info, dot_count = process_single_image(image_path)
                
                if annotated_image is None:
                    print(f"Failed to process {filename}")
                    continue
                
                # Get the nucleus mask file path
                mask_filename = f"{os.path.splitext(filename)[0]}_mask.tiff"
                nucleus_mask_path = os.path.join(nucleus_mask_folder, mask_filename)
                
                # Build JSON format for the image
                image_data = {
                    "image_name": os.path.splitext(filename)[0],
                    "label": label,
                    "nucleus_mask": nucleus_mask_path,
                    "dots": dot_info
                }
                
                # Save the annotated image as TIFF with maximum quality
                output_filename = os.path.join(output_folder, f"annotated_{filename}")
                # Save with original bit depth and compression
                cv2.imwrite(output_filename, 
                          annotated_image,
                          [cv2.IMWRITE_TIFF_COMPRESSION, 1])  # Use LZW compression for better quality
                
#                 results['images'][filename] = {
#                     'filename': filename,
#                     'annotated_filename': f"annotated_{filename}",
#                     'dot_count': dot_count,
#                     'dots': dot_info
#                 }
                
#                 results['total_images_processed'] += 1
                results.append(image_data)
    
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                continue
    
    with open(json_output_path, 'w') as f:
        json.dump(results, f, indent=4)
    
    return results

In [18]:
# Example usage:
input_folder = '/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/merged-pml-images-No-arsenic'
output_folder = '/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/merged-pml-images-No-arsenic-annotations'
json_output_path = '/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/merged-pml-images-No-arsenic-annotations/merged-pml-images-No-arsenic-annotations.json'
label = "not arsenic"  # Example label, change as needed
nucleus_mask_folder = "/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/input/binary_mask_nucleus_non_arsenic"

process_folder(input_folder, output_folder, json_output_path, label, nucleus_mask_folder)

Processing flattened_position_10_C1.tif...
Processing flattened_position_6_C1.tif...
Processing flattened_position_4_C1.tif...
Processing flattened_position_6-1_C1.tif...
Processing flattened_position_7-1_C1.tif...
Processing flattened_position_8_C1.tif...
Processing flattened_position_12_C1.tif...
Processing flattened_position_20-1_C1.tif...
Processing flattened_position_14-1_C1.tif...
Processing flattened_position_15-1_C1.tif...
Processing flattened_position_1-1_C1.tif...
Processing flattened_position_20_C1.tif...
Processing flattened_position_19-1_C1.tif...
Processing flattened_position_18-1_C1.tif...
Processing flattened_position_16_C1.tif...
Processing flattened_position_13-1_C1.tif...
Processing flattened_position_12-1_C1.tif...
Processing flattened_position_2_C1.tif...
Processing flattened_position_18_C1.tif...
Processing flattened_position_14_C1.tif...
Processing flattened_position_17-1_C1.tif...
Processing flattened_position_16-1_C1.tif...
Processing flattened_position_11_C1.t

[{'image_name': 'flattened_position_10_C1',
  'label': 'not arsenic',
  'nucleus_mask': '/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/input/binary_mask_nucleus_non_arsenic/flattened_position_10_C1_mask.tiff',
  'dots': [{'dot_id': 1,
    'position': {'x': 469, 'y': 511},
    'width': 3,
    'height': 1,
    'area': 0},
   {'dot_id': 2,
    'position': {'x': 484, 'y': 511},
    'width': 4,
    'height': 2,
    'area': 2},
   {'dot_id': 3,
    'position': {'x': 488, 'y': 506},
    'width': 3,
    'height': 3,
    'area': 3},
   {'dot_id': 4,
    'position': {'x': 490, 'y': 501},
    'width': 5,
    'height': 5,
    'area': 14},
   {'dot_id': 5,
    'position': {'x': 456, 'y': 496},
    'width': 1,
    'height': 1,
    'area': 0},
   {'dot_id': 6,
    'position': {'x': 475, 'y': 496},
    'width': 6,
    'height': 5,
    'area': 17},
   {'dot_id': 7,
    'position': {'x': 457, 'y': 492},
    'widt

In [20]:
# Example usage for arsenic:
input_folder = '/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/merged_pml_images_High_Arsenic'
output_folder = '/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/merged_pml_images_High_Arsenic_annotations'
json_output_path = '/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/merged_pml_images_High_Arsenic_annotations/merged_pml_images_High_Arsenic_annotations.json '
label = "arsenic"  # Example label, change as needed
nucleus_mask_folder = "/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/input/binary_mask_nucleus_high_arsenic"

process_folder(input_folder, output_folder, json_output_path, label, nucleus_mask_folder)

Processing flattened_position_10_C1.tif...
Processing flattened_position_6_C1.tif...
Processing flattened_position_34_C1.tif...
Processing flattened_position_26_C1.tif...
Processing flattened_position_38_C1.tif...
Processing flattened_position_4_C1.tif...
Processing flattened_position_8_C1.tif...
Processing flattened_position_12_C1.tif...
Processing flattened_position_24_C1.tif...
Processing flattened_position_28_C1.tif...
Processing flattened_position_36_C1.tif...
Processing flattened_position_32_C1.tif...
Processing flattened_position_20_C1.tif...
Processing flattened_position_16_C1.tif...
Processing flattened_position_22_C1.tif...
Processing flattened_position_30_C1.tif...
Processing flattened_position_2_C1.tif...
Processing flattened_position_18_C1.tif...
Processing flattened_position_14_C1.tif...
Processing flattened_position_35_C1.tif...
Processing flattened_position_39_C1.tif...
Processing flattened_position_27_C1.tif...
Processing flattened_position_11_C1.tif...
Processing flat

[{'image_name': 'flattened_position_10_C1',
  'label': 'arsenic',
  'nucleus_mask': '/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/input/binary_mask_nucleus_high_arsenic/flattened_position_10_C1_mask.tiff',
  'dots': [{'dot_id': 1,
    'position': {'x': 111, 'y': 470},
    'width': 1,
    'height': 1,
    'area': 0},
   {'dot_id': 2,
    'position': {'x': 257, 'y': 464},
    'width': 1,
    'height': 2,
    'area': 0},
   {'dot_id': 3,
    'position': {'x': 81, 'y': 462},
    'width': 5,
    'height': 4,
    'area': 8},
   {'dot_id': 4,
    'position': {'x': 264, 'y': 455},
    'width': 3,
    'height': 2,
    'area': 1},
   {'dot_id': 5,
    'position': {'x': 254, 'y': 453},
    'width': 1,
    'height': 1,
    'area': 0},
   {'dot_id': 6,
    'position': {'x': 95, 'y': 443},
    'width': 2,
    'height': 3,
    'area': 1},
   {'dot_id': 7,
    'position': {'x': 251, 'y': 439},
    'width': 5,
